#### Drzewa decyzyjne

Poniższy przykład opiera się na danych dotyczących zdolności kredytowej

In [3]:
import pandas as pd
data = pd.read_excel('practice_lab_3.xlsx')
columns = list(data.columns)
mask = data['Gender'].values == 'Female'
data['Gender'][mask] = 1
data ['Gender'][~mask] = 0
print(data)

cat_feature = pd.Categorical(X.Property_Area)
one_hot = pd.get_dummies(cat_feature)
data = pd.concat([data, one_hot], axis = 1)
data = data.drop(columns = ['Property_Area'])

    Gender Married  Dependents     Education Self_Employed  ApplicantIncome  \
0        0      No           0      Graduate            No             5849   
1        0     Yes           1      Graduate            No             4583   
2        0     Yes           0      Graduate           Yes             3000   
3        0     Yes           0  Not Graduate            No             2583   
4        0      No           0      Graduate            No             6000   
..     ...     ...         ...           ...           ...              ...   
609      1      No           0      Graduate            No             2900   
610      0     Yes           3      Graduate            No             4106   
611      0     Yes           1      Graduate            No             8072   
612      0     Yes           2      Graduate            No             7583   
613      1      No           0      Graduate           Yes             4583   

     CoapplicantIncome  LoanAmount  Loan_Amount_Ter

/tmp/ipykernel_43181/2507430838.py:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data['Gender'][mask] = 1
/tmp/ipykernel_43181/2507430838.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

Se